In [1]:
import pandas as pd
import numpy as np
import tensorflow
import keras

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
data = pd.read_csv('/Users/aaronmichaelbroderick/Desktop/Data Science/Data/devanagari.csv')

In [10]:
Y = data['character']

columns = list(data)
columns.remove('character')
X = data[columns]

In [11]:
# Change strings into numerical to work well with Keras

letters = list(set(Y))
Y_test = list(Y)

for i in range(len(letters)):
    for j in range(len(Y_test)):
        if letters[i] == Y_test[j]:
            Y_test[j] = i

In [12]:
results = pd.DataFrame()
results['cat'] = Y
results['num'] = Y_test

Y = results.num

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)

In [14]:
# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 46)
y_test = keras.utils.to_categorical(y_test, 46)

73600 train samples
18400 test samples


In [15]:
x_train = x_train.values
x_test = x_test.values

In [16]:
# input image dimensions, from our data
img_rows, img_cols = 32, 32
num_classes = 46

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (73600, 32, 32, 1)
73600 train samples
18400 test samples
Train on 73600 samples, validate on 18400 samples
Epoch 1/10
73600/73600 [==============================] - 172s 2ms/step - loss: 1.2450 - acc: 0.6527 - val_loss: 0.2744 - val_acc: 0.9224
Epoch 2/10
73600/73600 [==============================] - 165s 2ms/step - loss: 0.4461 - acc: 0.8665 - val_loss: 0.1669 - val_acc: 0.9510
Epoch 3/10
73600/73600 [==============================] - 160s 2ms/step - loss: 0.3189 - acc: 0.9037 - val_loss: 0.1308 - val_acc: 0.9614
Epoch 4/10
73600/73600 [==============================] - 160s 2ms/step - loss: 0.2533 - acc: 0.9223 - val_loss: 0.1117 - val_acc: 0.9676
Epoch 5/10
73600/73600 [==============================] - 159s 2ms/step - loss: 0.2148 - acc: 0.9336 - val_loss: 0.1062 - val_acc: 0.9691
Epoch 6/10
73600/73600 [==============================] - 389s 5ms/step - loss: 0.1840 - acc: 0.9429 - val_loss: 0.0948 - val_acc: 0.9718
Epoch 7/10
73600/73600 [=========================